In [5957]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import torch
from torch_geometric.data import Data
import torch_geometric.transforms as T
import torch.nn as nn
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch.nn import ModuleList
import random

In [5958]:
# 1. 걸음걸이 데이터
file_path = r"D:\K-DS\01.데이터\1.Training\원천데이터\1.걸음걸이\train_activity.csv"
Walk = pd.read_csv(file_path)
file_path = r"D:\K-DS\01.데이터\1.Training\라벨링데이터\1.걸음걸이\training_label.csv"
Walk_label = pd.read_csv(file_path)

In [5959]:
# 2. 수면 데이터
file_path = r'D:\K-DS\01.데이터\1.Training\원천데이터\2.수면\train_sleep.csv' 
Sleep = pd.read_csv(file_path)
file_path = r'D:\K-DS\01.데이터\1.Training\라벨링데이터\2.수면\training_label.csv' 
Sleep_label = pd.read_csv(file_path)

In [5960]:
# 3. 인지기능
file_path = r'D:\K-DS\01.데이터\1.Training\원천데이터\3.인지기능\train_mmse.csv' 
MMSE = pd.read_csv(file_path)
file_path = r'D:\K-DS\01.데이터\1.Training\라벨링데이터\3.인지기능\training_label.csv' 
MMSE_label = pd.read_csv(file_path)

In [5961]:
Walk_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Walk = pd.merge(Walk, Walk_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Walk['EMAIL'] = Walk['EMAIL'].str.extract(r'(\d{3})')

Sleep_label.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Sleep = pd.merge(Sleep, Sleep_label[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Sleep['EMAIL'] = Sleep['EMAIL'].str.extract(r'(\d{3})')

Walk

,EMAIL,activity_average_met,activity_cal_active,activity_cal_total,activity_class_5min,activity_daily_movement,activity_day_end,activity_day_start,activity_high,activity_inactive,...,activity_score_move_every_hour,activity_score_recovery_time,activity_score_stay_active,activity_score_training_frequency,activity_score_training_volume,activity_steps,activity_total,CONVERT(activity_class_5min USING utf8),CONVERT(activity_met_1min USING utf8),DIAG_NM
0,279,1.28125,196,2251,...,3353,2020-10-20T03:59:59+09:00,2020-10-19T04:00:00+09:00,0,714,...,100,100,54,100,97,4212,207,1/1/2/2/2/2/1/1/1/1/1/1/1/2/2/2/2/2/2/3/3/2/2/...,1.2/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,MCI
1,279,1.25000,145,2159,...,2516,2020-10-21T03:59:59+09:00,2020-10-20T04:00:00+09:00,0,719,...,100,100,58,96,96,3306,140,1/1/1/2/2/2/2/2/2/2/2/2/1/1/1/1/1/2/3/3/3/3/2/...,0.9/0.9/1.3/1/0.9/0.9/1.1/0.9/1/0.9/1/0.9/1/0....,MCI
2,279,1.21875,118,2140,...,1716,2020-10-22T03:59:59+09:00,2020-10-21T04:00:00+09:00,0,708,...,95,100,55,71,83,2550,131,1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/2/3/3/3/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,MCI
3,279,1.28125,180,2240,...,2791,2020-10-23T03:59:59+09:00,2020-10-22T04:00:00+09:00,0,705,...,100,100,58,71,59,4009,208,1/1/1/1/1/1/1/1/1/1/2/2/2/2/3/3/3/2/3/3/3/3/3/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,MCI
4,279,1.46875,374,2559,...,5393,2020-10-24T03:59:59+09:00,2020-10-23T04:00:00+09:00,0,437,...,100,100,89,40,46,8081,517,1/1/1/2/2/2/2/2/2/1/1/1/3/2/2/1/2/2/2/3/3/2/2/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,MCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,206,1.34375,249,2351,...,4411,2020-12-25T03:59:59+09:00,2020-12-24T04:00:00+09:00,1,780,...,43,100,45,40,95,4972,190,1/1/1/1/1/1/1/1/1/1/2/3/2/2/2/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,CN
9701,206,1.53125,570,2682,...,11057,2020-12-27T03:59:59+09:00,2020-12-26T04:00:00+09:00,1,518,...,95,100,76,71,93,11878,394,1/1/1/1/1/1/2/2/2/2/2/2/1/1/1/1/1/1/1/1/1/1/1/...,0.9/0.9/0.9/0.9/1.1/0.9/0.9/1.1/0.9/0.9/0.9/0....,CN
9702,206,1.34375,295,2331,...,5135,2020-12-28T03:59:59+09:00,2020-12-27T04:00:00+09:00,4,578,...,100,100,70,40,95,5752,207,1/1/1/2/3/2/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/1/...,1.1/0.9/0.9/0.9/1/0.9/0.9/0.9/0.9/0.9/0.9/0.9/...,CN
9703,206,1.31250,186,2280,...,3178,2020-12-29T03:59:59+09:00,2020-12-28T04:00:00+09:00,0,778,...,95,100,47,40,93,3852,156,1/1/1/1/1/1/1/1/1/1/1/1/2/2/0/0/0/0/0/0/0/0/0/...,0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0.9/0....,CN


In [5962]:
Walk = Walk.drop(['activity_class_5min', 'activity_met_1min', 
                    'CONVERT(activity_class_5min USING utf8)',
                    'CONVERT(activity_met_1min USING utf8)',
                    ], axis=1)

Sleep = Sleep.drop(['sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'sleep_is_longest',
                    'CONVERT(sleep_hr_5min USING utf8)',
                    'CONVERT(sleep_hypnogram_5min USING utf8)',
                    'CONVERT(sleep_rmssd_5min USING utf8)',
                    'sleep_temperature_delta',
                    ], axis=1)

In [5963]:
Walk['activity_day_start'] = pd.to_datetime(Walk['activity_day_start'])
Walk['activity_day_end'] = pd.to_datetime(Walk['activity_day_end'])
Walk['activity_time'] = Walk['activity_day_end'] - Walk['activity_day_start']

Walk.drop(['activity_day_start', 'activity_day_end', 'activity_time'], axis=1, inplace=True)

In [5964]:
Sleep['sleep_bedtime_start'] = pd.to_datetime(Sleep['sleep_bedtime_start'])
Sleep['sleep_bedtime_end'] = pd.to_datetime(Sleep['sleep_bedtime_end'])
Sleep['sleep_time'] = Sleep['sleep_bedtime_end'] - Sleep['sleep_bedtime_start']
Sleep['sleep_time'] = Sleep['sleep_time'].dt.total_seconds() / 60

Sleep.drop(['sleep_bedtime_start', 'sleep_bedtime_end'], axis=1, inplace=True)

In [5965]:
Walk['DIAG_NM'] = Walk['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2}).astype(int)
Sleep['DIAG_NM'] = Sleep['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2}).astype(int)
# 'DIAG_NM' 컬럼을 기준으로 행 개수 세기
label_counts = Sleep['DIAG_NM'].value_counts()

# 결과 출력
print(label_counts)

DIAG_NM
0    5781
1    3353
2     571
Name: count, dtype: int64


In [5966]:
MMSE.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
MMSE['EMAIL'] = MMSE['EMAIL'].str.extract(r'(\d{3})')
MMSE

,EMAIL,DIAG_SEQ,DIAG_NM,DOCTOR_NM,MMSE_NUM,MMSE_KIND,Q01,Q02,Q03,Q04,...,Q14_1,Q14_2,Q15,Q16_1,Q16_2,Q16_3,Q17,Q18,Q19,TOTAL
0,315,2,CN,-,2,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,29
1,014,3,MCI,-,3,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,28
2,220,2,CN,-,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,30
3,096,2,MCI,-,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,27
4,396,3,CN,-,3,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,181,1,CN,-,1,2,2,2,2,1,...,2,2,2,2,2,1,2,2,2,26
137,354,2,CN,-,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,26
138,448,2,MCI,-,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,28
139,394,3,CN,-,3,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,28


In [5967]:
# 2. 수면 + 3. 인지기능
Sleep = pd.merge(Sleep, MMSE[['EMAIL', 'TOTAL']], on='EMAIL', how='left')
Sleep.rename(columns={'TOTAL': 'total_mmse'}, inplace=True)
Sleep.drop(["EMAIL"], axis=1, inplace=True)
Sleep = Sleep.drop(['DIAG_NM'], axis=1)
Sleep

,sleep_awake,sleep_breath_average,sleep_deep,sleep_duration,sleep_efficiency,sleep_hr_average,sleep_hr_lowest,sleep_light,sleep_midpoint_at_delta,sleep_midpoint_time,...,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_deviation,sleep_total,sleep_time,total_mmse
0,8700,17.875,10110,37920,77,53.04,49,15330,84928,17820,...,100,78,65,89,56,94,0.00,29220,632.0,17
1,6570,18.000,7440,28680,77,52.82,49,11820,4582,12990,...,98,77,65,97,43,65,0.34,22110,478.0,17
2,10530,18.000,4620,32040,67,52.01,47,12360,7078,18390,...,94,60,41,75,68,62,-0.07,21510,534.0,17
3,4080,17.500,7440,25440,84,51.07,46,9060,5776,12480,...,98,80,81,83,73,62,-0.14,21360,424.0,17
4,7170,17.875,4320,20700,65,50.80,47,7470,-2220,9870,...,88,72,39,78,25,29,-0.50,13530,345.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,6000,15.875,1710,33360,82,63.75,57,19320,8017,16680,...,34,57,76,81,95,86,-0.09,27360,556.0,28
9701,5700,16.125,0,27720,79,67.40,61,16140,10129,14070,...,1,56,69,86,89,64,0.41,22020,462.0,28
9702,5070,15.625,480,35460,86,64.84,60,22170,10630,17490,...,9,59,86,89,98,97,0.27,30390,591.0,28
9703,2040,15.875,2220,27840,93,67.05,60,17880,4545,14040,...,44,59,98,89,86,80,-0.05,25800,464.0,28


In [5968]:
# 1. 걸음걸이 + (2. 수면 + 3. 인지기능)
ver1 = pd.concat([Walk, Sleep], axis=1)
#ver1.to_csv('ver1_combined_data.csv', index=False)

In [5969]:
# 값이 0인 행 중에서 2000개 랜덤 선택 후 삭제
rows_to_delete_0 = ver1[ver1['DIAG_NM'] == 0].sample(n=3000, random_state=42)
ver1 = ver1.drop(rows_to_delete_0.index)

# 값이 1인 행 중에서 1000개 랜덤 선택 후 삭제
rows_to_delete_1 = ver1[ver1['DIAG_NM'] == 1].sample(n=2000, random_state=42)
ver1 = ver1.drop(rows_to_delete_1.index)

# 결과 확인
print(ver1['DIAG_NM'].value_counts())

DIAG_NM
0    2781
1    1353
2     571
Name: count, dtype: int64


In [5970]:
Label = ver1[['DIAG_NM']].copy()
Label.rename(columns={'DIAG_NM': 'label'}, inplace=True)
print(Label)

      label
1         1
3         1
4         1
5         1
9         1
...     ...
9691      0
9693      0
9699      0
9701      0
9702      0

[4705 rows x 1 columns]


In [5971]:
ver1
ver1.drop(["EMAIL"], axis=1, inplace=True)
ver1.drop(["DIAG_NM"], axis=1, inplace=True)


In [5972]:
ver1

,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,activity_met_min_high,...,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_deviation,sleep_total,sleep_time,total_mmse
1,1.25000,145,2159,2516,0,719,0,131,9,0,...,98,77,65,97,43,65,0.34,22110,478.0,17
3,1.28125,180,2240,2791,0,705,0,203,5,0,...,98,80,81,83,73,62,-0.14,21360,424.0,17
4,1.46875,374,2559,5393,0,437,0,511,6,0,...,88,72,39,78,25,29,-0.50,13530,345.0,17
5,1.53125,436,2630,6860,0,377,0,548,13,0,...,98,75,95,70,71,74,-0.14,24330,453.0,17
9,1.50000,408,2587,6368,1,438,0,509,11,7,...,98,80,72,97,28,76,0.51,24780,516.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9691,1.59375,664,2752,13555,3,493,2,357,53,23,...,2,50,48,75,47,74,-0.02,24270,582.0,28
9693,1.37500,273,2374,4555,2,727,0,187,25,15,...,24,55,86,97,91,69,0.19,23070,447.0,28
9699,1.34375,227,2316,3863,3,735,2,155,17,21,...,53,46,74,97,84,82,-0.16,26400,546.0,28
9701,1.53125,570,2682,11057,1,518,1,332,61,7,...,1,56,69,86,89,64,0.41,22020,462.0,28


In [5973]:
scaler = StandardScaler()
ver1_normalized = scaler.fit_transform(ver1)
ver1_normalized_df = pd.DataFrame(ver1_normalized, columns=ver1.columns)

ver1_normalized_df

,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,activity_met_min_high,...,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_deviation,sleep_total,sleep_time,total_mmse
0,-1.088143,-1.145147,-0.931540,-1.137273,-0.440518,1.502988,-0.587206,-1.234520,-0.974262,-0.428583,...,0.687120,0.883349,-0.548304,1.135673,-0.399234,-0.328843,1.102450,-0.289828,-0.081880,-2.266259
1,-0.928780,-1.018678,-0.707893,-1.086964,-0.440518,1.402413,-0.587206,-0.647608,-1.064292,-0.428583,...,0.687120,1.109871,0.278893,0.237816,0.612196,-0.474540,-0.382227,-0.422054,-0.520907,-2.266259
2,0.027402,-0.317677,0.172888,-0.610942,-0.440518,-0.522874,-0.587206,1.863071,-1.041785,-0.428583,...,0.290026,0.505811,-1.892500,-0.082848,-1.006092,-2.077206,-1.495734,-1.802489,-1.163186,-2.266259
3,0.346129,-0.093646,0.368923,-0.342562,-0.440518,-0.953908,-0.587206,2.164679,-0.884232,-0.428583,...,0.687120,0.732334,1.002691,-0.595909,0.544768,0.108248,-0.382227,0.101559,-0.285133,-2.266259
4,0.186765,-0.194821,0.250197,-0.432571,-0.288998,-0.515690,-0.587206,1.846768,-0.929247,-0.280570,...,0.687120,1.109871,-0.186405,1.135673,-0.904949,0.205380,1.628273,0.180895,0.227065,-2.266259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,0.664856,0.730211,0.705774,0.882252,0.014041,-0.120575,2.106899,0.607732,0.016069,0.057744,...,-3.124979,-1.155352,-1.427201,-0.275246,-0.264376,0.108248,-0.011057,0.090981,0.763652,0.464596
4701,-0.450689,-0.682631,-0.337910,-0.764249,-0.137479,1.560459,-0.587206,-0.778033,-0.614142,-0.111413,...,-2.251373,-0.777815,0.537393,1.135673,1.219055,-0.134580,0.638488,-0.120580,-0.333914,0.464596
4702,-0.610053,-0.848848,-0.498052,-0.890847,0.014041,1.617930,2.106899,-1.038882,-0.794202,0.015455,...,-1.099801,-1.457381,-0.083005,1.135673,0.983054,0.496773,-0.444088,0.466502,0.470968,0.464596
4703,0.346129,0.390551,0.512499,0.425256,-0.288998,0.059023,0.759846,0.403943,0.196129,-0.280570,...,-3.164688,-0.702307,-0.341505,0.430214,1.151626,-0.377408,1.318965,-0.305695,-0.211962,0.464596


In [5974]:
# 피어슨 상관계수 행렬 계산
pearson_corr = ver1_normalized_df.T.corr(method='pearson')
print(pearson_corr)

threshold = np.percentile(pearson_corr.values.flatten(), 99)
edges = np.where((pearson_corr.values >= threshold) & (pearson_corr.values != 1))
edge_list = list(zip(edges[0], edges[1]))
print(len(edge_list))

# 엣지 리스트 확인
#print("생성된 엣지들 (상위 5% 이상):", edge_list)
# # 피어슨 상관계수 행렬을 Numpy 배열로 변환
# corr_array = pearson_corr.values

# # 대각선 요소 제외 (자기 자신과의 상관관계는 1이므로)
# np.fill_diagonal(corr_array, np.nan)

# # 행렬을 1차원 배열로 변환하고 NaN 값을 제외
# flattened_corr = corr_array.flatten()
# valid_corr = flattened_corr[~np.isnan(flattened_corr)]

          0         1         2         3         4         5         6      
0     1.000000  0.745040  0.132299  0.039592  0.354180  0.337087  0.448324  \
1     0.745040  1.000000  0.274094  0.369639  0.328238  0.648028  0.553701   
2     0.132299  0.274094  1.000000  0.549814  0.568611  0.265719  0.048596   
3     0.039592  0.369639  0.549814  1.000000  0.682991  0.590814  0.370141   
4     0.354180  0.328238  0.568611  0.682991  1.000000  0.267367  0.364055   
...        ...       ...       ...       ...       ...       ...       ...   
4700 -0.458350 -0.613355 -0.103647 -0.373386 -0.213057 -0.622357 -0.390584   
4701  0.147964  0.057548 -0.270694 -0.338493 -0.314217 -0.180590  0.069466   
4702  0.144246  0.047683 -0.290797 -0.312624 -0.278211 -0.192876  0.234297   
4703 -0.394254 -0.445535 -0.139542 -0.178917 -0.277869 -0.445995 -0.323815   
4704 -0.044326 -0.060789 -0.478279 -0.175263 -0.247178 -0.134136  0.241188   

          7         8         9     ...      4695      4696    

In [5975]:
# # 상위 95%에 해당하는 값 계산
# top_95_percent_value = np.percentile(valid_corr, 95)

# # 상위 95% 이상인 값들의 인덱스 구하기
# edge_indices = np.argwhere(corr_array >= top_95_percent_value)

# # 엣지 인덱스를 리스트로 변환
# edge_list = [[int(edge[0]), int(edge[1])] for edge in edge_indices]

# edge_list를 PyTorch 텐서로 변환
edge_tensor = torch.tensor(edge_list, dtype=torch.long)
edge_tensor = edge_tensor.T

In [5976]:
# 데이터프레임을 Numpy 배열로 변환
numpy_array = ver1_normalized_df.values

# Numpy 배열을 PyTorch 텐서로 변환
tensor_data = torch.tensor(numpy_array, dtype=torch.float32)

# 노드의 수 계산 (예: y의 길이로부터)
y = torch.tensor(Label['label'].values, dtype=torch.long)
num_nodes = y.size(0)
num_classes = y.unique().size(0)
# 모든 노드를 True로 설정
train_mask = torch.ones(num_nodes, dtype=torch.bool)
test_mask = torch.ones(num_nodes, dtype=torch.bool)

In [5977]:
# 결과 출력
print(f"PyTorch 텐서로 변환된 데이터프레임:\n{tensor_data[:5]}")  # 데이터의 첫 5개 행 출력
print(f"PyTorch 텐서로 변환된 엣지 리스트:\n{edge_tensor[:5]}")  # 엣지 리스트의 첫 5개 출력

num_nodes = y.size(0)
train_mask = torch.ones(num_nodes, dtype=torch.bool)
#test_mask = torch.ones(num_nodes, dtype=torch.bool)
#data = Data(x=tensor_data, edge_index=edge_tensor, y=y, train_mask=train_mask, test_mask=test_mask)
data = Data(x=tensor_data, edge_index=edge_tensor, y=y, train_mask=train_mask)
graph = data
print(data)

# split = T.RandomNodeSplit(num_val=0.2)
# graph = split(data)

PyTorch 텐서로 변환된 데이터프레임:
tensor([[-1.0881, -1.1451, -0.9315, -1.1373, -0.4405,  1.5030, -0.5872, -1.2345,
         -0.9743, -0.4286,  0.3463, -1.0656, -0.9253,  1.0997, -0.5506,  0.0489,
         -0.4117,  0.4073,  0.4694, -1.3173,  0.4677,  0.4228, -1.2272, -1.3758,
          0.2936,  0.9564,  0.9067, -0.0819, -0.5412, -1.0193, -0.8726, -0.6900,
         -0.4354, -0.4398,  0.2966,  0.0840, -0.3812, -1.0071,  0.2969,  0.1677,
          0.5188,  0.6871,  0.8833, -0.5483,  1.1357, -0.3992, -0.3288,  1.1024,
         -0.2898, -0.0819, -2.2663],
        [-0.9288, -1.0187, -0.7079, -1.0870, -0.4405,  1.4024, -0.5872, -0.6476,
         -1.0643, -0.4286,  0.6245, -0.6805, -1.0127,  0.6074, -0.7090, -0.8493,
         -0.9185,  0.4073,  0.4694, -1.3173, -0.2737, -1.2935, -1.1089, -0.9025,
         -0.4096,  0.5674,  0.9067, -0.5209,  0.2531, -1.2598, -1.3279, -1.2878,
         -0.3658, -0.5694, -0.3223,  1.8892,  0.5350, -1.3370,  0.6705,  0.3360,
          0.5188,  0.6871,  1.1099,  0.2789,  0.

In [5978]:
# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device="cpu"
graph = graph.to(device)

In [5979]:
class GAT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.convs = ModuleList([
            GATConv(data.num_node_features, 8, heads=8, dropout=0.6),
            GATConv(8 * 8, num_classes, heads=1, concat=False, dropout=0.6),
        ])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.6, training=self.training)
        return F.log_softmax(x, dim=-1)

In [5980]:
# 모델 학습 함수
def train_node_classifier(model, graph, optimizer, criterion, n_epochs=300):
    for epoch in range(1, n_epochs + 1):
        model.train() 
        optimizer.zero_grad() 
        out = model(graph)
        loss = criterion(out[graph.train_mask], graph.y[graph.train_mask]) 
        loss.backward() 
        optimizer.step() 

        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}')

    return model

In [5981]:
# 검증 함수
def eval_node_classifier(model, graph, mask):
    model.eval() 
    pred = model(graph).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

In [5982]:
# 모델 학습
gat = GAT().to(device)
optimizer_gat = torch.optim.Adam(gat.parameters(), lr=0.01, weight_decay=5e-4)
#optimizer_gat = torch.optim.Adam(gat.parameters(), lr=0.0004)

criterion = nn.CrossEntropyLoss()

In [5983]:
# 학습
trained_gat = train_node_classifier(gat, graph, optimizer_gat, criterion)

Epoch: 010, Train Loss: 0.853
Epoch: 020, Train Loss: 0.794
Epoch: 030, Train Loss: 0.763
Epoch: 040, Train Loss: 0.745
Epoch: 050, Train Loss: 0.732
Epoch: 060, Train Loss: 0.714
Epoch: 070, Train Loss: 0.691
Epoch: 080, Train Loss: 0.681
Epoch: 090, Train Loss: 0.662
Epoch: 100, Train Loss: 0.673
Epoch: 110, Train Loss: 0.658
Epoch: 120, Train Loss: 0.660
Epoch: 130, Train Loss: 0.637
Epoch: 140, Train Loss: 0.637
Epoch: 150, Train Loss: 0.638
Epoch: 160, Train Loss: 0.646
Epoch: 170, Train Loss: 0.635
Epoch: 180, Train Loss: 0.627
Epoch: 190, Train Loss: 0.626
Epoch: 200, Train Loss: 0.622
Epoch: 210, Train Loss: 0.623
Epoch: 220, Train Loss: 0.619
Epoch: 230, Train Loss: 0.623
Epoch: 240, Train Loss: 0.625
Epoch: 250, Train Loss: 0.614
Epoch: 260, Train Loss: 0.609
Epoch: 270, Train Loss: 0.603
Epoch: 280, Train Loss: 0.604
Epoch: 290, Train Loss: 0.595
Epoch: 300, Train Loss: 0.604


In [5984]:
# val_acc = eval_node_classifier(trained_gat, graph, graph.test_mask)
# print(f'Validation Accuracy: {val_acc:.3f}')

In [5985]:
# Validation Dataset
# 1. 걸음걸이 데이터
file_path = r"D:\K-DS\01.데이터\2.Validation\원천데이터\1.걸음걸이\val_activity.csv"
Walk_val = pd.read_csv(file_path)
file_path = r"D:\K-DS\01.데이터\2.Validation\라벨링데이터\1.걸음걸이\val_label.csv"
Walk_label_val = pd.read_csv(file_path)

# 2. 수면 데이터
file_path = r'D:\K-DS\01.데이터\2.Validation\원천데이터\2.수면\val_sleep.csv' 
Sleep_val = pd.read_csv(file_path)
file_path = r'D:\K-DS\01.데이터\2.Validation\라벨링데이터\2.수면\val_label.csv' 
Sleep_label_val = pd.read_csv(file_path)

# 3. 인지기능
file_path = r'D:\K-DS\01.데이터\2.Validation\원천데이터\3.인지기능\val_mmse.csv' 
MMSE_val = pd.read_csv(file_path)
file_path = r'D:\K-DS\01.데이터\2.Validation\라벨링데이터\3.인지기능\val_label.csv' 
MMSE_label_val = pd.read_csv(file_path)

In [5986]:
Walk_label_val.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Walk_val = pd.merge(Walk_val, Walk_label_val[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Walk_val['EMAIL'] = Walk_val['EMAIL'].str.extract(r'(\d{3})')

Sleep_label_val.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
Sleep_val = pd.merge(Sleep_val, Sleep_label_val[['EMAIL', 'DIAG_NM']], on='EMAIL', how='left')
Sleep_val['EMAIL'] = Sleep_val['EMAIL'].str.extract(r'(\d{3})')

In [5987]:
Walk_val = Walk_val.drop(['activity_class_5min', 'activity_met_1min', 
                    'CONVERT(activity_class_5min USING utf8)',
                    'CONVERT(activity_met_1min USING utf8)',
                    ], axis=1)

Sleep_val = Sleep_val.drop(['sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'sleep_is_longest',
                    'CONVERT(sleep_hr_5min USING utf8)',
                    'CONVERT(sleep_hypnogram_5min USING utf8)',
                    'CONVERT(sleep_rmssd_5min USING utf8)',
                    'sleep_temperature_delta',
                    ], axis=1)

In [5988]:
Walk_val['activity_day_start'] = pd.to_datetime(Walk_val['activity_day_start'])
Walk_val['activity_day_end'] = pd.to_datetime(Walk_val['activity_day_end'])
Walk_val['activity_time'] = Walk_val['activity_day_end'] - Walk_val['activity_day_start']

Walk_val.drop(['activity_day_start', 'activity_day_end', 'activity_time'], axis=1, inplace=True)

In [5989]:
Sleep_val['sleep_bedtime_start'] = pd.to_datetime(Sleep_val['sleep_bedtime_start'])
Sleep_val['sleep_bedtime_end'] = pd.to_datetime(Sleep_val['sleep_bedtime_end'])
Sleep_val['sleep_time'] = Sleep_val['sleep_bedtime_end'] - Sleep_val['sleep_bedtime_start']
Sleep_val['sleep_time'] = Sleep_val['sleep_time'].dt.total_seconds() / 60

Sleep_val.drop(['sleep_bedtime_start', 'sleep_bedtime_end'], axis=1, inplace=True)

In [5990]:
Walk_val['DIAG_NM'] = Walk_val['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2}).astype(int)
Sleep_val['DIAG_NM'] = Sleep_val['DIAG_NM'].replace({'CN': 0, 'MCI': 1, 'Dem': 2}).astype(int)
# 'DIAG_NM' 컬럼을 기준으로 행 개수 세기
label_counts = Sleep_val['DIAG_NM'].value_counts()

# 결과 출력
print(label_counts)

DIAG_NM
0    1956
1     308
2     214
Name: count, dtype: int64


In [5991]:
Label_val = Sleep_val[['DIAG_NM']].copy()
Label_val.rename(columns={'DIAG_NM': 'label'}, inplace=True)
print(Label_val)

      label
0         0
1         0
2         0
3         0
4         0
...     ...
2473      0
2474      0
2475      0
2476      0
2477      0

[2478 rows x 1 columns]


In [5992]:
Walk_val.drop(["EMAIL"], axis=1, inplace=True)
Walk_val = Walk_val.drop(['DIAG_NM'], axis=1)
Sleep_val = Sleep_val.drop(['DIAG_NM'], axis=1)
print(Walk_val.columns)

Index(['activity_average_met', 'activity_cal_active', 'activity_cal_total',
       'activity_daily_movement', 'activity_high', 'activity_inactive',
       'activity_inactivity_alerts', 'activity_low', 'activity_medium',
       'activity_met_min_high', 'activity_met_min_inactive',
       'activity_met_min_low', 'activity_met_min_medium', 'activity_non_wear',
       'activity_rest', 'activity_score', 'activity_score_meet_daily_targets',
       'activity_score_move_every_hour', 'activity_score_recovery_time',
       'activity_score_stay_active', 'activity_score_training_frequency',
       'activity_score_training_volume', 'activity_steps', 'activity_total'],
      dtype='object')


In [5993]:
MMSE_val.rename(columns={'SAMPLE_EMAIL': 'EMAIL'}, inplace=True)
MMSE_val['EMAIL'] = MMSE_val['EMAIL'].str.zfill(3)
MMSE_val['EMAIL'] = MMSE_val['EMAIL'].str.extract(r'(\d{3})')
MMSE_val

,EMAIL,DIAG_SEQ,DIAG_NM,DOCTOR_NM,MMSE_NUM,MMSE_KIND,Q01,Q02,Q03,Q04,...,Q14_1,Q14_2,Q15,Q16_1,Q16_2,Q16_3,Q17,Q18,Q19,TOTAL
0,163,4,CN,-,4,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,27
1,280,1,CN,-,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,29
2,322,1,CN,-,1,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,28
3,173,2,CN,-,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,30
4,320,3,CN,-,3,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,29
5,263,6,CN,-,6,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,30
6,027,2,CN,-,2,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,27
7,424,2,CN,-,2,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,25
8,266,5,CN,-,5,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,27
9,245,1,MCI,-,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,30


In [5994]:
# 2. 수면 + 3. 인지기능
Sleep_val = pd.merge(Sleep_val, MMSE_val[['EMAIL', 'TOTAL']], on='EMAIL', how='left')
Sleep_val.rename(columns={'TOTAL': 'total_mmse'}, inplace=True)
Sleep_val.drop(["EMAIL"], axis=1, inplace=True)
Sleep_val

,sleep_awake,sleep_breath_average,sleep_deep,sleep_duration,sleep_efficiency,sleep_hr_average,sleep_hr_lowest,sleep_light,sleep_midpoint_at_delta,sleep_midpoint_time,...,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_deviation,sleep_total,sleep_time,total_mmse
0,13950,16.500,2100,35460,61,70.53,62,15030,6693,17280,...,41,50,27,97,66,62,-0.12,21510,591.0,29
1,7680,17.375,2430,24720,69,68.01,63,12210,10495,11490,...,49,43,46,91,35,44,-0.32,17040,412.0,29
2,11880,17.000,2820,33240,64,64.89,59,15780,5439,17460,...,56,47,34,89,41,62,0.07,21360,554.0,29
3,9600,17.250,1770,29700,68,63.70,57,16350,7241,15750,...,35,40,44,86,29,56,-0.41,20100,495.0,29
4,10710,17.250,3150,31440,66,67.07,60,14010,6037,17220,...,64,55,39,14,54,59,-0.27,20730,524.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,1680,18.500,6030,13980,88,63.33,60,4800,7751,7080,...,97,90,90,89,22,25,-0.27,12300,233.0,29
2474,12840,18.000,4020,41640,69,60.22,53,18270,10561,18600,...,81,75,46,97,95,92,-0.13,28800,694.0,29
2475,11250,18.375,6060,38280,71,65.38,59,16830,9520,17910,...,97,56,51,81,62,85,0.21,27030,638.0,29
2476,7410,18.000,7110,43320,83,62.43,55,21150,7352,22890,...,98,71,79,75,98,100,0.69,35910,722.0,29


In [5995]:
# 1. 걸음걸이 + (2. 수면 + 3. 인지기능)
ver1_val = pd.concat([Walk_val, Sleep_val], axis=1)
#ver1.to_csv('ver1_combined_data.csv', index=False)

In [5996]:
ver1_val

,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,activity_met_min_high,...,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_deviation,sleep_total,sleep_time,total_mmse
0,1.71875,730,2944,14346,0,417,0,545,47,0,...,41,50,27,97,66,62,-0.12,21510,591.0,29
1,1.40625,342,2449,6352,0,473,0,392,8,0,...,49,43,46,91,35,44,-0.32,17040,412.0,29
2,1.46875,401,2544,7297,0,586,0,362,24,0,...,56,47,34,89,41,62,0.07,21360,554.0,29
3,0.34375,27,1850,491,0,176,0,34,1,0,...,35,40,44,86,29,56,-0.41,20100,495.0,29
4,1.46875,333,2518,5861,0,646,0,377,6,0,...,64,55,39,14,54,59,-0.27,20730,524.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,1.59375,648,2760,12256,0,444,0,381,66,0,...,97,90,90,89,22,25,-0.27,12300,233.0,29
2474,1.43750,379,2507,6973,0,623,0,361,13,0,...,81,75,46,97,95,92,-0.13,28800,694.0,29
2475,1.56250,670,2747,13008,0,422,0,366,76,0,...,97,56,51,81,62,85,0.21,27030,638.0,29
2476,1.46875,556,2570,11043,0,452,2,244,68,0,...,98,71,79,75,98,100,0.69,35910,722.0,29


In [5997]:
scaler = StandardScaler()
ver1_val_normalized = scaler.fit_transform(ver1_val)
ver1_val_normalized_df = pd.DataFrame(ver1_val_normalized, columns=ver1_val.columns)

ver1_val_normalized_df

,activity_average_met,activity_cal_active,activity_cal_total,activity_daily_movement,activity_high,activity_inactive,activity_inactivity_alerts,activity_low,activity_medium,activity_met_min_high,...,sleep_score_deep,sleep_score_disturbances,sleep_score_efficiency,sleep_score_latency,sleep_score_rem,sleep_score_total,sleep_temperature_deviation,sleep_total,sleep_time,total_mmse
0,1.641784,1.240628,1.531695,1.369189,-0.384153,-0.912701,-0.688825,2.500439,0.019742,-0.417109,...,-1.597688,-1.408273,-2.436204,1.141694,0.165495,-0.603831,-0.358504,-0.510968,0.898931,0.378748
1,-0.182584,-0.364049,-0.037897,-0.363210,-0.384153,-0.491607,-0.688825,1.152866,-0.917324,-0.417109,...,-1.286412,-2.000522,-1.462516,0.746014,-0.892601,-1.550053,-1.008135,-1.448510,-0.787823,0.378748
2,0.182290,-0.120039,0.263338,-0.158417,-0.384153,0.358100,-0.688825,0.888636,-0.532887,-0.417109,...,-1.014047,-1.662094,-2.077477,0.614120,-0.687808,-0.603831,0.258647,-0.542429,0.550273,0.378748
3,-6.385436,-1.666816,-1.937263,-1.633361,-0.384153,-2.724909,-0.688825,-2.000279,-1.085515,-0.417109,...,-1.831144,-2.254343,-1.565010,0.416279,-1.097394,-0.919238,-1.300470,-0.806703,-0.005697,0.378748
4,0.182290,-0.401271,0.180895,-0.469616,-0.384153,0.809273,-0.688825,1.020751,-0.965379,-0.417109,...,-0.702772,-0.985238,-1.821243,-4.331892,-0.244090,-0.761534,-0.845727,-0.674566,0.267576,0.378748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0.912037,0.901495,0.948251,0.916260,-0.384153,-0.709674,-0.688825,1.055982,0.476261,-0.417109,...,0.581239,1.976007,0.792339,0.614120,-1.336319,-2.548842,-0.845727,-2.442682,-2.474577,0.378748
2474,-0.000147,-0.211026,0.146015,-0.228632,-0.384153,0.636323,-0.688825,0.879828,-0.797188,-0.417109,...,-0.041312,0.706902,-1.462516,1.141694,1.155328,0.973206,-0.390985,1.018044,1.869522,0.378748
2475,0.729600,0.992482,0.907029,1.079228,-0.384153,-0.875103,-0.688825,0.923867,0.716534,-0.417109,...,0.581239,-0.900631,-1.206283,0.086545,0.028967,0.605231,0.713389,0.646802,1.341822,0.378748
2476,0.182290,0.521005,0.345781,0.653388,-0.384153,-0.649517,1.515060,-0.150669,0.524315,-0.417109,...,0.620148,0.368474,0.228625,-0.309136,1.257724,1.393749,2.272506,2.509302,2.133371,0.378748


In [5998]:
# 피어슨 상관계수 행렬 계산
pearson_corr_val = ver1_val_normalized_df.T.corr(method='pearson')
print(pearson_corr_val)

threshold = np.percentile(pearson_corr_val.values.flatten(), 98)
edges_val = np.where((pearson_corr_val.values >= threshold) & (pearson_corr_val.values != 1))
edge_list_val = list(zip(edges_val[0], edges_val[1]))
print(len(edge_list_val))

          0         1         2         3         4         5         6      
0     1.000000  0.694748  0.729437 -0.014222  0.411298  0.453640  0.372124  \
1     0.694748  1.000000  0.730803  0.346246  0.412815  0.457482  0.629034   
2     0.729437  0.730803  1.000000  0.188352  0.651113  0.802801  0.431904   
3    -0.014222  0.346246  0.188352  1.000000  0.036461  0.169091  0.138500   
4     0.411298  0.412815  0.651113  0.036461  1.000000  0.549483  0.286404   
...        ...       ...       ...       ...       ...       ...       ...   
2473  0.142569  0.381697 -0.010914 -0.135298  0.032849 -0.347007  0.355379   
2474  0.440558  0.136460  0.552320  0.039307  0.324323  0.638094  0.070489   
2475  0.708155  0.386604  0.473746  0.158824  0.172609  0.169215  0.175735   
2476 -0.042820 -0.408867 -0.151710  0.092159 -0.201266 -0.082983 -0.405990   
2477  0.043831  0.406676  0.109080  0.361821  0.115059  0.044771  0.185526   

          7         8         9     ...      2468      2469    

In [5999]:
# edge_list를 PyTorch 텐서로 변환
edge_tensor_val = torch.tensor(edge_list_val, dtype=torch.long)
edge_tensor_val = edge_tensor_val.T

In [6000]:
# 데이터프레임을 Numpy 배열로 변환
numpy_array_val = ver1_val_normalized_df.values

# Numpy 배열을 PyTorch 텐서로 변환
tensor_data_val = torch.tensor(numpy_array_val, dtype=torch.float32)

# 노드의 수 계산 (예: y의 길이로부터)
y_val = torch.tensor(Label_val['label'].values, dtype=torch.long)
num_nodes_val = y_val.size(0)
num_classes_val = y_val.unique().size(0)
# 모든 노드를 True로 설정
train_mask = torch.ones(num_nodes_val, dtype=torch.bool)
test_mask = torch.ones(num_nodes_val, dtype=torch.bool)

In [6001]:
# 결과 출력
print(f"PyTorch 텐서로 변환된 데이터프레임:\n{tensor_data[:5]}")  # 데이터의 첫 5개 행 출력
print(f"PyTorch 텐서로 변환된 엣지 리스트:\n{edge_tensor[:5]}")  # 엣지 리스트의 첫 5개 출력

data_val = Data(x=tensor_data_val, edge_index=edge_tensor_val, y=y_val, train_mask=train_mask, test_mask=test_mask)
graph_val = data_val
print(data_val)

PyTorch 텐서로 변환된 데이터프레임:
tensor([[-1.0881, -1.1451, -0.9315, -1.1373, -0.4405,  1.5030, -0.5872, -1.2345,
         -0.9743, -0.4286,  0.3463, -1.0656, -0.9253,  1.0997, -0.5506,  0.0489,
         -0.4117,  0.4073,  0.4694, -1.3173,  0.4677,  0.4228, -1.2272, -1.3758,
          0.2936,  0.9564,  0.9067, -0.0819, -0.5412, -1.0193, -0.8726, -0.6900,
         -0.4354, -0.4398,  0.2966,  0.0840, -0.3812, -1.0071,  0.2969,  0.1677,
          0.5188,  0.6871,  0.8833, -0.5483,  1.1357, -0.3992, -0.3288,  1.1024,
         -0.2898, -0.0819, -2.2663],
        [-0.9288, -1.0187, -0.7079, -1.0870, -0.4405,  1.4024, -0.5872, -0.6476,
         -1.0643, -0.4286,  0.6245, -0.6805, -1.0127,  0.6074, -0.7090, -0.8493,
         -0.9185,  0.4073,  0.4694, -1.3173, -0.2737, -1.2935, -1.1089, -0.9025,
         -0.4096,  0.5674,  0.9067, -0.5209,  0.2531, -1.2598, -1.3279, -1.2878,
         -0.3658, -0.5694, -0.3223,  1.8892,  0.5350, -1.3370,  0.6705,  0.3360,
          0.5188,  0.6871,  1.1099,  0.2789,  0.

In [6002]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
graph_val = graph_val.to(device)
val_acc = eval_node_classifier(trained_gat, graph_val, graph_val.test_mask)
print(f'Validation Accuracy: {val_acc:.3f}')

Validation Accuracy: 0.813
